# User interface Node #

In [1]:
#! /usr/bin/env python3

import rospy
import time
import sys
import select
import os   
import actionlib
import actionlib.msg
import assignment_2_2022.msg
from nav_msgs.msg import Odometry
from assignment_2_2022.srv import Ngoal
import ipywidgets as widgets
from ipywidgets import VBox, HBox, Layout
import logging
from IPython.display import display
import jupyros as jr
#from jupyros import ros3d

act_clnt = None
goal = assignment_2_2022.msg.PlanningGoal() 
goal.target_pose.pose.position.z = 0.0
info = Ngoal()
srv_info = None
gf = 0 # goal flag to increase the number of goal only the first time the state pass to succeeded
Rpx = widgets.FloatText(value = '0.00', disabled = True, layout = L2, continuous_update = True)
Rpy = widgets.FloatText(value = '0.00', disabled = True, layout = L2, continuous_update = True)

NameError: name 'L2' is not defined

In [ ]:
def D_cb(b):
    if ((act_clnt.get_state()) == 1):
        act_clnt.cancel_goal() # delete the goal
        n_deleted = rospy.get_param('n_deleted') # get the actual number of goal deleted
        n_deleted = n_deleted + 1 # increase it
        rospy.set_param('n_deleted', n_deleted) # load it in the paramserver

In [ ]:
def N_cb(b):
    if ((Gx.disabled == True) and (Gy.disabled == True)):
        Gx.disabled = False
        Gy.disabled = False     
        N.disabled = True

In [ ]:
def NGx_cb(value):   
    Gx.disabled = True
    goal.target_pose.pose.position.x = float(Gx.value)
    if (Gy.disabled):
        N.disabled = False
        act_clnt.send_goal(goal)
    

In [ ]:
def NGy_cb(value):    
    Gy.disabled = True
    goal.target_pose.pose.position.y = float(Gy.value)
    if (Gx.disabled):
        N.disabled = False  
        act_clnt.send_goal(goal)

In [ ]:
def I_cb(b):
    info = srv_info()
    Gr.value = info.n_goal
    Gd.value = info.n_deleted

In [ ]:
def UI(): 
    global D, N, I, Gx, Gy, Gr, Gd, Rpx, Rpy
    
    # different layout:
    L1 = Layout(width = 'auto', height = '10%', min_width = '80px', min_height = '54px', max_width = '160px')
    L2 = Layout(width = 'auto', height = '5%', min_width = '40px', min_height = '20px', max_width = '80px')
    
    # delete goal button:
    D = widgets.Button(value = False, description = 'Delete Goal', disabled = False, button_style = 'danger', layout = L1, continuous_update = False)
    
    # new goal button:
    N = widgets.Button(value = False, description = 'New Goal', disabled = False, button_style = 'success', layout = L1, continuous_update = False)
    
    # information button:
    I = widgets.Button(value = False, description = 'Information', disabled = False, button_style = 'info', layout = L1, continuous_update = False)
    
    # info box for the goal coordinates:
    # name:
    Gx_n = widgets.Text(value = 'x_goal', disabled = True, layout = L2)
    Gy_n = widgets.Text(value = 'y_goal', disabled = True, layout = L2)
    # data:
    Gx = widgets.FloatText(value = '0.00', disabled = True, layout = L2, continuous_update = False)
    Gy = widgets.FloatText(value = '0.00', disabled = True, layout = L2, continuous_update = False)
    # combination:
    Gx_c = VBox([Gx_n, Gx])
    Gy_c = VBox([Gy_n, Gy])
    Gxy = HBox([Gx_c, Gy_c])
    
    # info box for the robot position:
    # name:
    Rpx_n = widgets.Text(value = 'x_robot', disabled = True, layout = L2)
    Rpy_n = widgets.Text(value = 'y_robot', disabled = True, layout = L2)
    # data:
    #Rpx = widgets.FloatText(value = '0.00', disabled = True, layout = L2, continuous_update = True)
    #Rpy = widgets.FloatText(value = '0.00', disabled = True, layout = L2, continuous_update = True)
    # combination:
    Rpx_c = VBox([Gx_n, Gx])
    Rpy_c = VBox([Gy_n, Gy])
    Rpxy = HBox([Gx_c, Gy_c])
    
    
    # info box for the goal reached and deleted: FloatText
    # name:
    Gr_n = widgets.Text(value = 'goal_r', disabled = True, layout = L2)
    Gd_n = widgets.Text(value = 'goal_d', disabled = True, layout = L2)
    # data:
    Gr = widgets.FloatText(value = '0.00', disabled = True, layout = L2, continuous_update = False)
    Gd = widgets.FloatText(value = '0.00', disabled = True, layout = L2, continuous_update = False)
    # combination:
    Gr_c = VBox([Gr_n, Gr])
    Gd_c = VBox([Gd_n, Gd])
    Grd = HBox([Gr_c, Gd_c])
    
    # combine everything
    UI = VBox([D, HBox([N, Gxy]), HBox([I, Grd, Rpxy])])
    
    # callbacks:
    D.on_click(D_cb)
    N.on_click(N_cb)
    Gx.observe(NGx_cb, names = 'value')
    Gy.observe(NGy_cb, names = 'value')
    I.on_click(I_cb)
    
    display(UI)


In [ ]:
def init():
    global goal, act_clnt, info, srv_info
    
    rospy.init_node('user_interface', anonymous=True) # init the node
    sub_odom = jr.subscribe('/odom', Odometry, odom_cb)
    act_clnt = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)
    act_clnt.wait_for_server()
    srv_info = rospy.ServiceProxy("/info", Ngoal)
    rospy.wait_for_service("/info")
    rospy.set_param('n_goal', 0)
    rospy.set_param('n_deleted', 0)
    
    

In [ ]:
def odom_cb(msg):
    global Rpx, Rpy
    Rpx.value = msg.pose.pose.position.x
    Rpy.value = msg.pose.pose.position.y

In [ ]:
def grafic_interface():
    v = ros3d.Viewer()
    rc = ros3d.ROSConnection(url = "ws://localhost:9090")
    tf_client = ros3d.TFClient(ros = rc, fixed_frame = 'map')
    
    laser_view = ros3d.LaserScan(topic = "/scan", ros = rc, tf_client = tf_client)
    urdf = ros3d.URDFModel(ros = rc, tf_client = tf_client)
    g = ros3d.GridModel()
    v.objects = [g, laser_view, urdf]
    
    display(v)

In [ ]:
def check_goal():
    global gf
    
    if (act_clnt.get_state() == 3 and gf == 0): # if i've reached the goal
        n_goal = rospy.get_param('n_goal') # get the actual number of goal
        n_goal = n_goal + 1 # increase it
        rospy.set_param('n_goal', n_goal) # load it in the paramserver
        gf = 1
    elif (act_clnt.get_state() != 3):
        gf = 0    

In [ ]:
def main():
    time.sleep(1)
    
    init()    
    check_goal()
    UI()  
    

if __name__ == "__main__":
    main()